<a href="https://colab.research.google.com/github/ashwin-yedte/visual-intelligence-travel-finance/blob/main/notebooks/Visual%20Intelligence%20Layer/destination_matching.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**VLM INTELLIGENCE LAYER**

STEP 2: DESTINATION MATCHING
Match each user image separately against VL Encoding database

Features:
- Load VL encoding embeddings (47 destinations)
- Match each user image independently
- Get top-10 destinations per image
- Track theme distribution
- Prepare for theme aggregation in Step 3

# =================================================================
 PREREQUISITES
# =================================================================

MUST RUN BEFORE THIS:
1. Step 1 cells (creates user_embeddings.npy)
2. VL Encoding must be complete in Google Drive

# =================================================================
Step 1: CONFIGURATION
# =================================================================


In [ ]:
print("="*80)
print("STEP 2: DESTINATION MATCHING - VLM INTELLIGENCE LAYER")
print("="*80)

class Step2Config:
    """Configuration for Step 2"""

    # Paths (Google Drive)
    BASE_PATH = '/content/drive/MyDrive/visual-intelligence-travel-finance'
    EMBEDDINGS_PATH = BASE_PATH + '/data/vl_encoding/embeddings'
    PROMPTS_PATH = BASE_PATH + '/data/vl_encoding/prompts'
    METADATA_PATH = BASE_PATH + '/data/landmarks/metadata.json'

    # Matching parameters
    TOP_K_PER_IMAGE = 10
    MIN_SIMILARITY_SCORE = 0.20

    # Output
    OUTPUT_FILE = "step2_matches.json"

print("Configuration loaded")
print("="*80)

STEP 2: DESTINATION MATCHING - VLM INTELLIGENCE LAYER
Configuration loaded


# =================================================================
Step 2: MOUNT GOOGLE DRIVE
# =================================================================


In [ ]:
print("\n" + "="*80)
print("MOUNTING GOOGLE DRIVE")
print("="*80)

from google.colab import drive
drive.mount('/content/drive')

print("Google Drive mounted")
print("="*80)



MOUNTING GOOGLE DRIVE
Mounted at /content/drive
Google Drive mounted


# =================================================================
Step 3: LOAD VL ENCODING DATABASE
# =================================================================


In [ ]:
print("\n" + "="*80)
print("LOADING VL ENCODING DATABASE")
print("="*80)

import numpy as np
import json
from typing import Dict, List, Any

# Load destination embeddings
print("Loading embeddings...")
vl_data = np.load(Step2Config.EMBEDDINGS_PATH + '/all_embeddings.npz')
destination_ids = vl_data['destination_ids']
destination_embeddings = vl_data['destination_embeddings']

print("Loaded " + str(len(destination_ids)) + " destination embeddings")
print("Embedding shape: " + str(destination_embeddings.shape))

# Load embedding index
print("\nLoading embedding index...")
with open(Step2Config.EMBEDDINGS_PATH + '/embedding_index.json', 'r') as f:
    embedding_index = json.load(f)
print("Embedding index loaded")

# Load destination prompts
print("\nLoading destination prompts...")
with open(Step2Config.PROMPTS_PATH + '/destination_prompts.json', 'r') as f:
    destination_prompts = json.load(f)
print("Loaded prompts for " + str(len(destination_prompts)) + " destinations")

# Load metadata
print("\nLoading metadata...")
with open(Step2Config.METADATA_PATH, 'r') as f:
    metadata = json.load(f)
print("Metadata loaded: " + str(metadata['total_destinations']) + " destinations")

print("\n" + "="*80)
print("VL DATABASE LOADED SUCCESSFULLY")
print("="*80)



LOADING VL ENCODING DATABASE
Loading embeddings...
Loaded 168 destination embeddings
Embedding shape: (168, 512)

Loading embedding index...
Embedding index loaded

Loading destination prompts...
Loaded prompts for 168 destinations

Loading metadata...
Metadata loaded: 168 destinations

VL DATABASE LOADED SUCCESSFULLY


# =================================================================
Step 4: HELPER FUNCTIONS
# =================================================================


In [ ]:
def get_destination_theme(dest_id: str) -> str:
    """
    Get theme for a destination from metadata.

    Args:
        dest_id: Destination ID (e.g., 'BEACH_GOA_AGONDA_BEACH')

    Returns:
        Theme name (e.g., 'Beach', 'Waterfall')
    """

    for theme in metadata['themes']:
        for state in theme['states']:
            for destination in state['destinations']:
                if destination['destination_id'] == dest_id:
                    return theme['theme_name']

    return 'Unknown'


def get_destination_info(dest_id: str) -> Dict[str, Any]:
    """
    Get full destination information from metadata.

    Returns:
        Dictionary with name, state, theme, etc.
    """

    for theme in metadata['themes']:
        for state in theme['states']:
            for destination in state['destinations']:
                if destination['destination_id'] == dest_id:
                    return {
                        'destination_id': dest_id,
                        'name': destination['destination_name'],
                        'state': state['state_name'],
                        'theme': theme['theme_name'],
                        'image_count': destination.get('image_count', 0),
                        'geo_location': destination.get('geo_location', {}),
                        'offers': destination.get('offers', {})
                    }

    return {
        'destination_id': dest_id,
        'name': 'Unknown',
        'state': 'Unknown',
        'theme': 'Unknown',
        'image_count': 0,
        'geo_location': {},
        'offers': {}
    }


print("Helper functions loaded")
print("="*80)


Helper functions loaded


# =================================================================
Step 5: DESTINATION MATCHING FUNCTION
# =================================================================


In [ ]:
def match_destinations_per_image(user_embeddings: np.ndarray) -> Dict[str, Any]:
    """
    Match each user image separately against all destinations.

    This is the core of Step 2:
    - Takes each user image embedding
    - Compares with all 47 destination embeddings
    - Returns top-K matches per image
    - Tracks theme distribution

    Args:
        user_embeddings: Array of shape (N, 512) where N is number of images

    Returns:
        Dictionary with per-image matches and statistics
    """

    print("\n" + "="*80)
    print("MATCHING DESTINATIONS PER IMAGE")
    print("="*80)

    num_images = len(user_embeddings)
    print("Number of user images: " + str(num_images))
    print("Number of destinations: " + str(len(destination_ids)))
    print("Top-K per image: " + str(Step2Config.TOP_K_PER_IMAGE))

    per_image_matches = {}
    all_matched_destinations = set()
    theme_counter = {}

    for img_idx in range(num_images):
        image_key = "image_" + str(img_idx + 1)
        user_embedding = user_embeddings[img_idx]

        print("\n" + "-"*80)
        print("Processing " + image_key)
        print("-"*80)

        # Normalize user embedding
        user_embedding = user_embedding / np.linalg.norm(user_embedding)

        # Compute similarities with all destinations
        similarities = np.dot(destination_embeddings, user_embedding)

        # Get top-K indices
        top_indices = np.argsort(similarities)[::-1][:Step2Config.TOP_K_PER_IMAGE]

        # Build matches for this image
        matches = []

        for rank, dest_idx in enumerate(top_indices, 1):
            dest_id = destination_ids[dest_idx]
            score = float(similarities[dest_idx])

            # Skip if below threshold
            if score < Step2Config.MIN_SIMILARITY_SCORE:
                continue

            # Get destination info
            dest_info = get_destination_info(dest_id)
            theme = dest_info['theme']

            # Track theme
            if theme not in theme_counter:
                theme_counter[theme] = 0
            theme_counter[theme] += 1

            # Track destination
            all_matched_destinations.add(dest_id)

            # Build match object
            match = {
                'rank': rank,
                'destination_id': dest_id,
                'destination_name': dest_info['name'],
                'state': dest_info['state'],
                'theme': theme,
                'similarity_score': round(score * 100, 2),
                'raw_score': score
            }

            matches.append(match)

            if rank <= 3:
                print("  " + str(rank) + ". " + dest_info['name'] +
                      " (" + theme + ") - " + str(round(score * 100, 1)) + "%")

        per_image_matches[image_key] = {
            'matches': matches,
            'num_matches': len(matches),
            'top_theme': matches[0]['theme'] if matches else 'Unknown',
            'avg_score': round(np.mean([m['raw_score'] for m in matches]) * 100, 2) if matches else 0
        }

    print("\n" + "="*80)
    print("MATCHING COMPLETE")
    print("="*80)
    print("Total unique destinations matched: " + str(len(all_matched_destinations)))
    print("Theme distribution:")
    for theme, count in sorted(theme_counter.items(), key=lambda x: x[1], reverse=True):
        print("  " + theme + ": " + str(count) + " matches")
    print("="*80)

    return {
        'per_image_matches': per_image_matches,
        'theme_distribution': theme_counter,
        'total_unique_destinations': len(all_matched_destinations),
        'num_images_processed': num_images
    }


print("Matching function loaded")
print("="*80)


Matching function loaded


# =================================================================
Step 6: SAVE STEP 2 OUTPUTS
# =================================================================


In [ ]:
def save_step2_outputs(matching_output: Dict) -> None:
    """
    Save Step 2 outputs for use in Step 3.

    Saves:
    - step2_matches.json: Full matching results
    """

    print("\n" + "="*80)
    print("SAVING STEP 2 OUTPUTS")
    print("="*80)

    # Convert to JSON-serializable format
    output_data = {
        'per_image_matches': matching_output['per_image_matches'],
        'theme_distribution': matching_output['theme_distribution'],
        'total_unique_destinations': matching_output['total_unique_destinations'],
        'num_images_processed': matching_output['num_images_processed']
    }

    # Save to JSON
    output_path = '/content/' + Step2Config.OUTPUT_FILE
    with open(output_path, 'w') as f:
        json.dump(output_data, f, indent=2)

    print("Saved to: " + output_path)
    print("="*80)
    print("\nReady for Step 3: Theme Extraction and Aggregation")
    print("="*80)


print("Save function loaded")
print("="*80)

Save function loaded


# =================================================================
Step 7: MAIN EXECUTION FUNCTION
# =================================================================


In [ ]:
def run_step2(user_embeddings_path: str = '/content/user_embeddings.npy'):
    """
    Complete Step 2 execution.

    Args:
        user_embeddings_path: Path to user embeddings from Step 1

    Returns:
        Matching results dictionary
    """

    print("\n" + "="*80)
    print("EXECUTING STEP 2: DESTINATION MATCHING")
    print("="*80)

    # Load user embeddings from Step 1
    print("\nLoading user embeddings from Step 1...")
    try:
        user_embeddings = np.load(user_embeddings_path)
        print("Loaded user embeddings: " + str(user_embeddings.shape))
    except FileNotFoundError:
        print("ERROR: User embeddings not found at " + user_embeddings_path)
        print("Please run Step 1 first to generate user embeddings")
        return None

    # Match destinations
    matching_output = match_destinations_per_image(user_embeddings)

    # Save outputs
    save_step2_outputs(matching_output)

    return matching_output


print("Main execution function loaded")
print("="*80)
print("\nSTEP 2 INITIALIZED - Ready to match destinations")
print("="*80)
print("\nTO RUN:")
print("  result = run_step2()")
print("="*80)

Main execution function loaded

STEP 2 INITIALIZED - Ready to match destinations

TO RUN:
  result = run_step2()
